In [1]:
import lzma
import pickle

import dill
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
from jitr.optical_potentials import chuq

In [3]:
import rxmc

Using database version X4-2024-12-31 located in: /mnt/home/beyerkyl/x4db/unpack_exfor-2024/X4-2024-12-31


## read in CHUQ parameter samples

In [4]:
neutron = (1, 0)
proton = (1, 1)

In [5]:
chuq_params = chuq.get_samples_federal()

In [7]:
nparams = len(chuq_params[0])

## read in observations and models

In [8]:
from pathlib import Path

evidence_dir = Path("../evidence/data")
model_dir = Path("../model/")

output_dir = Path("./chuq_propagation/")
output_dir.mkdir(parents=True, exist_ok=True)

In [9]:
with lzma.open(evidence_dir / "nn_elastic_observations.xz", "rb") as f:
    nn_elastic_obs = pickle.load(f)
with lzma.open(evidence_dir / "pp_elastic_observations.xz", "rb") as f:
    pp_elastic_obs = pickle.load(f)

In [10]:
with open(model_dir / "chuq_nn_model.pkl", "rb") as f:
    nn_model = dill.load(f)

In [11]:
with open(model_dir / "chuq_pp_model.pkl", "rb") as f:
    pp_model = dill.load(f)

## propagate KDUQ into constraint observables

In [14]:
yth_nn = []
for entry_id, obs in tqdm(nn_elastic_obs.items()):
    for x in obs:
        yth_nn.append(
            np.array(
                [
                    nn_model.visualizable_model_prediction(x, *p)
                    for p in chuq_params
                ]
            )
        )

100%|██████████| 18/18 [00:32<00:00,  1.80s/it]


In [15]:
yth_pp = []
for entry_id, obs in tqdm(pp_elastic_obs.items()):
    for x in obs:
        yth_pp.append(
            np.array(
                [
                    pp_model.visualizable_model_prediction(x, *p)
                    for p in chuq_params
                ]
            )
        )

100%|██████████| 55/55 [01:10<00:00,  1.28s/it]


## write posterior predictive distributions to disk

In [16]:
with open(output_dir / "nn_chuq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [17]:
with open(output_dir / "pp_chuq_posterior_predictive.pkl", "wb") as f:
    pickle.dump(yth_pp, f)

## Propagate posteriors from our fitting

In [ ]:
results_full = np.load("../calibration/chuq_cal_prod/result.npz")
n_chains, n_steps, n_params = results_full["model_chains"].shape
(n_chains, n_steps, n_params)

In [ ]:
def draw_samples(x, low_cut=None, high_cut=None, n_samples=None):
    chains, steps, n_params = x.shape
    if low_cut is None:
        low_cut = 0
    if high_cut is None:
        high_cut = steps - 1
    samples = x[:, low_cut:high_cut, ...].reshape(
        (chains * (high_cut - low_cut), n_params)
    )
    if n_samples is None:
        return samples
    else:
        return samples[
            np.random.choice(np.arange(chains * (high_cut - low_cut)), n_samples), ...
        ]

In [ ]:
posterior = draw_samples(results_full["model_chains"], n_samples=1000)

In [ ]:
yth_nn = []
for entry_id, obs in tqdm(nn_elastic_obs.items()):
    for x in obs:
        yth_nn.append(
            np.array(
                [
                    nn_model.visualizable_model_prediction(x, *p)
                    for p in posterior
                ]
            )
        )

In [ ]:
yth_pp = []
for entry_id, obs in tqdm(pp_elastic_obs.items()):
    for x in obs:
        yth_pp.append(
            np.array(
                [
                    pp_model.visualizable_model_prediction(x, *p)
                    for p in posterior
                ]
            )
        )

## write posterior predictive distributions to disk

In [ ]:
with open(output_dir / "nn_chuq_cal.pkl", "wb") as f:
    pickle.dump(yth_nn, f)

In [ ]:
with open(output_dir / "pp_chuq_cal.pkl", "wb") as f:
    pickle.dump(yth_pp, f)